# Notebook 4: Armor

In this notebook, we explore developments to `battlesim` from version 0.3.5 including modifications with `Terrain` object and how terrains now impact the simulation. Further to this, we also incorporate `armor` calculations to all of our units.

### Requirements:

- `numpy`
- `pandas`
- `matplotlib`
- `scipy.stats`
- `numba`

In [1]:
import numpy as np
import sys
sys.path.insert(0,"../")
# import
import battlesim as bsm

As before, we create a `Battle` object. Note that by default we will use the clone wars example now, removing the file dependency:

In [2]:
bat = bsm.Battle()
bat

bsm.Battle(init=False)

In [3]:
bat.db_

,Allegiance,Type,Armor,HP,Damage,Dmg Speed,Range,Movement Speed,Accuracy,Miss,Shield,Shield Regen,allegiance_int
Name,,,,,,,,,,,,,
local militia,Republic,Standard,15,20,10,1.00,2.0,0.6,5,30,0,0.00,0
b1 battledroid,CIS,Standard,35,20,15,1.00,2.5,0.5,30,35,0,0.00,1
clone trooper,Republic,Standard,50,35,16,1.20,3.0,0.7,65,75,0,0.00,0
b2 battledroid,CIS,Standard,100,40,20,1.20,3.5,0.4,70,40,0,0.00,1
arc trooper,Republic,Elite,100,100,20,0.90,5.0,1.0,90,95,0,0.00,0
bx-series droid commando,CIS,Elite,70,50,22,1.30,7.0,0.8,70,95,0,0.00,1
clone sharpshooter,Republic,Specialist,50,40,50,0.20,15.0,0.2,60,40,0,0.00,0
battle droid assassin,CIS,Specialist,60,20,50,0.15,15.0,0.2,55,45,0,0.00,1
clone commando,Republic,Elite,100,120,20,1.50,3.0,0.9,97,97,30,0.20,0


## Existense of Armor within `M_` attribute

In [5]:
# creates 3 army groups
AA = list(zip(
    np.random.choice(np.asarray(bat.db_.index), 3),
    np.random.randint(1,50,3))
)
# assigns
bat.create_army(AA)

bsm.Battle(init=True, n_armies=3, simulated=False)

In [6]:
bat.M_["armor"]

array([400., 400., 400., 400., 400., 400., 400., 400., 400., 400., 400.,
       400., 400., 400., 400., 400., 400.,  15.,  15.,  15.,  15.,  15.,
        15.,  15.,  15.,  15.,  15.,  15.,  70.,  70.,  70.,  70.,  70.,
        70.,  70.,  70.,  70.,  70.,  70.,  70.,  70.,  70.,  70.,  70.,
        70.,  70.,  70.,  70.,  70.,  70.,  70.,  70.,  70.,  70.,  70.,
        70.,  70.,  70.,  70.,  70.,  70.,  70.,  70.,  70.,  70.,  70.,
        70.,  70.,  70.,  70.,  70.,  70.,  70.], dtype=float32)

## The `armor` attribute

Armor exists as a parameter that goes $\alpha_i \in [0, \infty]$, and damage calculations delivered against unit $j$ from attacking unit $i$ (from 0.3.4) move from:

\begin{align}
H_j - d_i
\end{align}

to

\begin{align}
H_j - d_i \left(\frac{z_i - z_j}{2} + 1 \right)
\end{align}

where $d_i$ is the base damage, $z_i$ and $z_j$ is the height at which unit $i$ and $j$ sit, respectively. Further to this, if armor rating $\alpha_j > 0$, then:

\begin{align}
\alpha_j - d_i \left(\frac{z_i - z_j}{2} + 1 \right), \quad \alpha_j > 0
\end{align}

where the right-hand side is $<= 0$, the difference is subtracted away from $H_j$. Thus our final *modus operandi* becomes:

\begin{align}
\alpha_j - d_i \left(\frac{z_i - z_j}{2} + 1 \right), \quad \alpha_j > 0 \\
H_j - d_i \left(\frac{z_i - z_j}{2} + 1 \right), \quad \alpha_j <= 0 \\
\end{align}

In future versions, we may adjust this to reduce the amount of damage done to armor vs health, consider options such as *pierce damage*, which can do more to armor, etc.